In [2]:
import os 
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
import time
import re
import warnings
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings('ignore')

In [31]:
# create a folder with the name of data
os.makedirs('data', exist_ok=True)

In [33]:
# Load dataset from Hugging Face Datasets and store the output column after extracting the python code into a csv file 
ds = load_dataset("jtatman/python-code-dataset-500k")
ds = pd.DataFrame(ds['train'])
ds = ds.drop(['instruction', 'system'], axis=1)
ds['English_code'] = ds['output'].apply(lambda x: re.search(r'```python(.*?)```', x, re.DOTALL).group(1) if re.search(r'```python(.*?)```', x, re.DOTALL) else None)
ds = ds.drop(['output'], axis=1)
ds.to_csv('data/python_code_dataset.csv', index=False)
ds.head()

,English_code
0,\nfor i in range(10): # First digit\n for ...
1,\ndef count_distinct_states(matrix):\n coun...
2,\ndef remove_spaces_and_punctuation(s):\n r...
3,\nimport math\n\ndef is_prime(n):\n # Check...
4,"\nclass String:\n def __init__(self, string..."


In [35]:
class Config:
    def __init__(self, max_rows=None):
        self.input_path = 'data/python_code_dataset.csv'
        self.output_path = 'data/google_code_translations.csv'
        self.checkpoint_path = 'data/translation_checkpoint.json'
        self.keywords_path = 'data/segregated_data.csv'
        self.batch_size = 5
        self.source_lang = 'en'
        self.target_lang = 'hi'
        self.sleep_time = 0.2
        self.max_retries = 3
        self.max_rows = max_rows

class CheckpointManager:
    """Manages saving and loading of translation progress"""
    def __init__(self, checkpoint_path: str):
        self.checkpoint_path = checkpoint_path
        self.processed_indices = set()
        self._load_checkpoint()

    def _load_checkpoint(self) -> None:
        """Load existing checkpoint if available"""
        if os.path.exists(self.checkpoint_path):
            try:
                with open(self.checkpoint_path, 'r') as f:
                    data = json.load(f)
                self.processed_indices = set(data.get('processed_indices', []))
                print(f"Loaded checkpoint with {len(self.processed_indices)} processed items")
            except Exception as e:
                print(f"Error loading checkpoint: {str(e)}")
                self.processed_indices = set()

    def save_checkpoint(self) -> None:
        """Save current progress to checkpoint file"""
        try:
            data = {'processed_indices': list(self.processed_indices)}
            with open(self.checkpoint_path, 'w') as f:
                json.dump(data, f)
        except Exception as e:
            print(f"Error saving checkpoint: {str(e)}")

    def mark_processed(self, index: int) -> None:
        """Mark an item as processed and save checkpoint"""
        self.processed_indices.add(index)
        self.save_checkpoint()

    def is_processed(self, index: int) -> bool:
        """Check if an item has been processed"""
        return index in self.processed_indices

    def get_unprocessed_indices(self, total_items: int) -> List[int]:
        """Get list of indices that haven't been processed yet"""
        return [i for i in range(total_items) if not self.is_processed(i)]

class KeywordManager:
    """Manages programming keyword translations"""
    def __init__(self, keywords_path: str):
        self.keywords_path = keywords_path
        self.keywords = self._load_keywords()
        self._add_special_cases()

    def _load_keywords(self) -> Dict[str, str]:
        """Load keyword translations from file"""
        try:
            df = pd.read_csv(self.keywords_path)
            # Drop non-Hindi translations
            columns_to_drop = [
                'FrenchKey.txt', 'SpanishKey.txt', 'KurdishKey.txt',
                'BengaliKey.txt', 'MandarinKey.txt', 'GreekKey.txt'
            ]
            df.drop(columns=columns_to_drop, inplace=True)
            df.dropna(inplace=True)
            return {row['EnglishKey.txt']: row['HindiKey.txt'] for _, row in df.iterrows()}
        except Exception as e:
            print(f"Error loading keywords: {str(e)}")
            return {}

    def _add_special_cases(self) -> None:
        """Add special case translations"""
        special_cases = {
            'i': 'ई',
            'j': 'जे',
            'k': 'के'
        }
        self.keywords.update(special_cases)

    def get_translation(self, word: str) -> Optional[str]:
        """Get translation for a keyword if available"""
        return self.keywords.get(word)

class CodeDataset(Dataset):
    """Dataset for code translation"""
    def __init__(self, codes: List[str], indices: List[int]):
        self.codes = codes
        self.indices = indices

    def __len__(self) -> int:
        return len(self.codes)

    def __getitem__(self, idx: int) -> Dict[str, any]:
        return {
            'index': self.indices[idx],
            'code': self.codes[idx]
        }

def custom_collate(batch: List[Dict]) -> Dict[str, List]:
    """Custom collate function for DataLoader"""
    return {
        'indices': [item['index'] for item in batch],
        'codes': [item['code'] for item in batch]
    }

class CodeTranslator:
    def __init__(self, config: Config, keyword_manager: KeywordManager):
        self.config = config
        self.keyword_manager = keyword_manager
        self.translator = GoogleTranslator(
            source=config.source_lang,
            target=config.target_lang
        )
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

    def process_compound_word(self, word: str) -> str:
        """Handle translation of compound words with underscores"""
        if '_' in word:
            parts = word.split('_')
            translated_parts = []
            for part in parts:
                translated = self.safe_translate(part)
                # If translation contains space, replace with underscore
                translated = translated.replace(' ', '_') if translated else part
                translated_parts.append(translated)
            return '_'.join(translated_parts)
        return word

    def translate_token(self, token: str) -> str:
        if token.isspace():
            return token
        elif '_' in token:
            parts = token.split('_')
            translated_parts = []
            for part in parts:
                if part:
                    keyword_trans = self.keyword_manager.get_translation(part)
                    if keyword_trans:
                        translated_parts.append(keyword_trans)
                    else:
                        trans = self.safe_translate(part)
                        if ' ' in trans:
                            trans = trans.replace(' ', '_')
                        translated_parts.append(trans)
            return '_'.join(translated_parts)
        elif token.isalpha():
            keyword_trans = self.keyword_manager.get_translation(token)
            if keyword_trans:
                return keyword_trans.replace(' ', '_')
            translation = self.safe_translate(token)
            return translation.replace(' ', '_')
        return token

    def safe_translate(self, text: str) -> str:
        if not text or not isinstance(text, str):
            return text

        for attempt in range(self.config.max_retries):
            try:
                translated = self.translator.translate(text)
                if ' ' in translated:
                    translated = translated.replace(' ', '_')
                if any(c.isascii() and c.isalpha() for c in translated):
                    translated = self.translator.translate(text.lower()).replace(' ', '_')
                return translated
            except Exception as e:
                if attempt == self.config.max_retries - 1:
                    return text
        return text

    def translate_line(self, line: str) -> str:
        indent = len(line) - len(line.lstrip())
        line = line.lstrip()

        if not line:
            return line

        try:
            if '#' in line:
                code_part, comment_part = line.split('#', 1)
                translated_comment = self.safe_translate(comment_part.strip())

                if code_part:
                    tokens = re.findall(r'[a-zA-Z_]+|\d+|[^\w\s]|\s+', code_part)
                    translated_tokens = [self.translate_token(token) for token in tokens]
                    translated_code = ''.join(translated_tokens)
                    return ' ' * indent + translated_code.rstrip() + ' #' + translated_comment
                return ' ' * indent + '#' + translated_comment

            tokens = re.findall(r'[a-zA-Z_]+|\d+|[^\w\s]|\s+', line)
            translated_tokens = [self.translate_token(token) for token in tokens]
            return ' ' * indent + ''.join(translated_tokens)

        except Exception as e:
            print(f"Line translation error: {str(e)}")
            return line
    def translate_code(self, code: str) -> str:
        if not isinstance(code, str):
            return ""

        if '\\n' in code:
            lines = code.strip("'\"").split('\\n')
            translated_lines = [self.translate_line(line.strip()) for line in lines]
            return '\\n '.join(translated_lines)

        lines = code.split('\n')
        translated_lines = [self.translate_line(line) for line in lines]
        return '\n'.join(translated_lines)

    def process_batch(self, batch: Dict[str, List]) -> Tuple[List[int], List[str]]:
        """Process a batch of code samples"""
        indices = batch['indices']
        codes = batch['codes']

        translated_batch = []
        for code in codes:
            if isinstance(code, torch.Tensor):
                code = code.cpu().numpy().item()
            translated_code = self.translate_code(code)
            translated_batch.append(translated_code)

        return indices, translated_batch

class TranslationManager:
    """Manages the overall translation process"""
    def __init__(self, config: Config):
        self.config = config
        self.checkpoint_manager = CheckpointManager(config.checkpoint_path)
        self.keyword_manager = KeywordManager(config.keywords_path)
        self.translator = CodeTranslator(config, self.keyword_manager)

    def prepare_data(self) -> Tuple[pd.DataFrame, List[int]]:
        if os.path.exists(self.config.output_path):
            results_df = pd.read_csv(self.config.output_path)
            input_df = pd.read_csv(self.config.input_path)
        else:
            input_df = pd.read_csv(self.config.input_path)
            if self.config.max_rows:
                input_df = input_df.head(self.config.max_rows)
            results_df = pd.DataFrame({
                'English_code': input_df['English_code'],
                'Hindi_code': [None] * len(input_df)
            })

        unprocessed_indices = self.checkpoint_manager.get_unprocessed_indices(len(input_df))
        if self.config.max_rows:
            unprocessed_indices = unprocessed_indices[:self.config.max_rows]
        return results_df, unprocessed_indices

    def process_translations(self) -> Optional[pd.DataFrame]:
        """Process all translations with checkpointing"""
        try:
            results_df, unprocessed_indices = self.prepare_data()

            if not unprocessed_indices:
                print("All items have been processed!")
                return results_df

            print(f"Found {len(unprocessed_indices)} unprocessed items")

            # Create dataset and dataloader
            unprocessed_codes = [
                results_df.iloc[i]['English_code'] for i in unprocessed_indices
            ]
            dataset = CodeDataset(unprocessed_codes, unprocessed_indices)
            dataloader = DataLoader(
                dataset,
                batch_size=self.config.batch_size,
                shuffle=False,
                collate_fn=custom_collate
            )

            # Process batches
            try:
                with tqdm(total=len(unprocessed_indices), desc="Translating code") as pbar:
                    for batch in dataloader:
                        indices, translated_codes = self.translator.process_batch(batch)

                        # Update results and save progress
                        for idx, translated_code in zip(indices, translated_codes):
                            results_df.at[idx, 'Hindi_code'] = translated_code
                            self.checkpoint_manager.mark_processed(idx)

                        # Save intermediate results
                        results_df.to_csv(self.config.output_path, index=False)
                        pbar.update(len(indices))

            except KeyboardInterrupt:
                print("\nProcess interrupted by user. Saving progress...")
                results_df.to_csv(self.config.output_path, index=False)
                return results_df

            print(f"\nProcessing completed! Results saved to: {self.config.output_path}")
            return results_df

        except Exception as e:
            print(f"Error during processing: {str(e)}")
            traceback.print_exc()
            if 'results_df' in locals():
                results_df.to_csv(self.config.output_path, index=False)
                return results_df
            return None


"""Main entry point"""
# Create config
config = Config(max_rows=50)

# Create data directory if it doesn't exist
Path('data').mkdir(exist_ok=True)

# Clean up GPU memory if available
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Initialize and run translation manager
manager = TranslationManager(config)
processed_df = manager.process_translations()

if processed_df is not None:
    print("\nProcessing completed successfully!")


Using device: cpu
Found 50 unprocessed items


Translating code: 100%|██████████| 50/50 [06:35<00:00,  7.91s/it]


Processing completed! Results saved to: data/google_code_translations.csv

Processing completed successfully!


In [ ]:
API_key = "YOUR_API_KEY"

In [ ]:
from openai import OpenAI
import pandas as pd
import re
from tqdm import tqdm

class Config:
    def __init__(self, max_rows=None, example_count = 5):
        self.input_path = 'data/google_code_translations.csv'
        self.output_path = 'data/translations_gpt.csv'
        self.checkpoint_path = 'data/checkpoint_gpt.json'
        self.max_rows = max_rows
        self.batch_size = 5
        self.openai_api_key = API_key
        self.examples_count = example_count
        self.client = OpenAI(api_key=self.openai_api_key)

class KeywordReplacer:
    def __init__(self):
        self.keywords = self._load_keywords()
        
    def _load_keywords(self):
        df = pd.read_csv('data/segregated_data.csv')
        columns_to_drop = [
            'FrenchKey.txt', 'SpanishKey.txt', 'KurdishKey.txt',
            'BengaliKey.txt', 'MandarinKey.txt', 'GreekKey.txt'
        ]
        df.drop(columns=columns_to_drop, inplace=True)
        df.dropna(inplace=True)
        return {row['EnglishKey.txt']: row['HindiKey.txt'] for _, row in df.iterrows()}
    
    def replace_keywords(self, code):
        # Split code into tokens while preserving structure
        tokens = re.findall(r'[a-zA-Z_]+|\d+|[^\w\s]|\s+', code)
        translated_tokens = []
        
        for token in tokens:
            if token in self.keywords:
                translated_tokens.append(self.keywords[token])
            elif token == 'True':
                translated_tokens.append('सत्य')
            elif token == 'False':
                translated_tokens.append('असत्य')
            else:
                translated_tokens.append(token)
                
        return ''.join(translated_tokens)

class GPTTranslator:
    def __init__(self, config):
        self.config = config
        self.examples = self.load_examples()
        self.keyword_replacer = KeywordReplacer()

    def load_examples(self):
        df = pd.read_csv(self.config.input_path)
        return df.head(self.config.examples_count)[['English_code', 'Hindi_code']]

    def create_prompt(self, code_to_translate):
        examples_text = ""
        for i, row in self.examples.iterrows():
            examples_text += f"\n\nExample {i+1}:\n"
            examples_text += f"English code:\n{row['English_code']}\n"
            examples_text += f"Hindi translated code:\n{row['Hindi_code']}\n------------------------\n"
            
        prompt = f"""Complete the translation of this partially English Python code to completely Hindi python code:
        - Translate variable names, function names, strings and comments to Hindi
        - Join multi-word Hindi translations with underscores
        - Break down compound English words separated by underscores and translate each part into sensible Hindi and join them back with underscores
        - Preserve code structure and syntax
        - Here are some examples of translations:
    
        {examples_text}
        
        Now translate partially translated code to completely in Hindi:
        {code_to_translate}"""
        
        return prompt

    def translate_code(self, code):
        # First replace known keywords
        partially_translated = self.keyword_replacer.replace_keywords(code)

        # Then use GPT to complete the translation
        prompt = self.create_prompt(partially_translated)
        try:
            # print(f"Prompt:\n{prompt}")
            response = self.config.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a Expert Python code translator who understands the nuanses of language in coding and converts code from English to  Hindi code while preserving functionality. Return only the translated code without any explanation."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )
            translated_code = response.choices[0].message.content.strip()

            # Clean up the response to extract just the code
            if "```python" in translated_code:
                translated_code = translated_code.split("```python")[1].split("```")[0].strip()
            elif "```" in translated_code:
                translated_code = translated_code.split("```")[1].strip()
                
            return translated_code
        except Exception as e:
            print(f"Translation error: {str(e)}")
            return code

def run_translation(max_rows=None):
    config = Config(max_rows=max_rows, example_count=5)
    translator = GPTTranslator(config)
    
    df = pd.read_csv(config.input_path)
    if max_rows:
        df = df.iloc[config.examples_count:max_rows+config.examples_count]
    
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        translated = translator.translate_code(row['English_code'])
        results.append({
            'English_code': row['English_code'],
            'Hindi_code': translated
        })
        
    results_df = pd.DataFrame(results)
    results_df.to_csv(config.output_path, index=False)
    return results_df

# Usage
translated_df = run_translation(max_rows=5)

100%|██████████| 5/5 [00:17<00:00,  3.43s/it]


In [46]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from tqdm import tqdm
import re


def keyword_reverse_translation(code, reverse_keywords):
    tokens = re.findall(r'[\u0900-\u097F_]+|[a-zA-Z_]+|\d+|[^\w\s]|\s+', code)
    translated_tokens = []
    
    for token in tokens:
        if token in reverse_keywords:
            translated_tokens.append(reverse_keywords[token])
        else:
            translated_tokens.append(token)
    
    return ''.join(translated_tokens)

class TranslationEvaluator:
    def __init__(self, config):
        self.config = config
        self.keyword_replacer = KeywordReplacer()
        nltk.download('punkt')
        self.reverse_keywords = {v: k for k, v in self.keyword_replacer.keywords.items()}

    def reverse_translate_code(self, hindi_code):
        # First replace known keywords
        partially_translated = keyword_reverse_translation(hindi_code, self.reverse_keywords)
        
        prompt = f"""Complete the translation of this partially translated Python code to English:
        - The code already has Python keywords translated to English
        - Translate remaining variable names and comments
        - Convert Hindi compound words (with underscores) to appropriate English terms
        - Preserve code structure and syntax
        
        Partially translated code:
        {partially_translated}"""
        
        try:
            response = self.config.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a Python code translator converting Hindi code to English."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )
            return self.clean_code(response.choices[0].message.content.strip())
        except Exception as e:
            print(f"Reverse translation error: {str(e)}")
            return hindi_code
    def clean_code(self, code):
        """Remove markdown and normalize code"""
        if "```python" in code:
            code = code.split("```python")[1].split("```")[0].strip()
        elif "```" in code:
            code = code.split("```")[1].strip()
        return code.strip()

    def calculate_bleu(self, original, translated):
        """Calculate BLEU score between original and translated code"""
        smooth = SmoothingFunction().method1
        
        # Tokenize the code
        def tokenize(code):
            return nltk.word_tokenize(code)
        
        reference = [tokenize(original)]
        candidate = tokenize(translated)
        
        return sentence_bleu(reference, candidate, smoothing_function=smooth)

    def evaluate_translations(self, df):
        """Evaluate translations using round-trip and BLEU score"""
        results = []
        
        for _, row in tqdm(df.iterrows(), desc="Evaluating translations"):
            original = row['English_code']
            hindi = row['Hindi_code']
            
            # Round-trip translation
            back_translated = self.reverse_translate_code(hindi)
            
            # Calculate BLEU score
            bleu_score = self.calculate_bleu(original, back_translated)
            
            results.append({
                'original': original,
                'hindi': hindi,
                'back_translated': back_translated,
                'bleu_score': bleu_score
            })
            
        return pd.DataFrame(results)

# Usage example
def run_evaluation():
    config = Config()
    evaluator = TranslationEvaluator(config)
    
    # Load translations
    df = pd.read_csv('data/translations_gpt.csv')
    
    # Run evaluation
    results = evaluator.evaluate_translations(df)
    
    # Save results
    results.to_csv('data/evaluation_results.csv', index=False)
    
    # Print average BLEU score
    avg_bleu = results['bleu_score'].mean()
    print(f"Average BLEU Score: {avg_bleu:.4f}")
    
    return results

# Run evaluation
evaluation_results = run_evaluation()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ankitkumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Evaluating translations: 5it [00:12,  2.59s/it]

Average BLEU Score: 0.8150


In [3]:
def load_keywords():
    keywords = pd.read_csv('data/segregated_data.csv')
    keywords.drop(columns=['FrenchKey.txt','SpanishKey.txt','KurdishKey.txt','BengaliKey.txt','MandarinKey.txt','GreekKey.txt'], inplace=True)
    keywords.dropna(inplace=True)
    keywords_dict = {row['EnglishKey.txt']: row['HindiKey.txt'] for _, row in keywords.iterrows()}
    return keywords_dict

In [4]:
keywords = load_keywords()

In [5]:
keywords

{'abs': 'निरपेक्ष_मान',
 'all': 'सब',
 'any': 'कोई',
 'ascii': 'आस्की',
 'bin': 'द्वि',
 'bool': 'बूल',
 'bytearray': 'बाइटरे',
 'bytes': 'बाइट्स',
 'callable': 'बुलाने_योग्य',
 'chr': 'अक्षर',
 'classmethod': 'वर्ग_विधि',
 'compile': 'संकलन',
 'complex': 'समिश्र',
 'delattr': 'गुणमिट',
 'dict': 'कोश',
 'dir': 'निर्देशिका',
 'divmod': 'भाग_मापांक',
 'enumerate': 'गणना',
 'eval': 'आंकें',
 'exec': 'अमल',
 'filter': 'छानें',
 'float': 'दश',
 'format': 'प्रारूप',
 'frozenset': 'जमासेट',
 'getattr': 'गुणप्राप्त',
 'globals': 'वैश्विक',
 'hasattr': 'गुणहै',
 'hash': 'हैश',
 'help': 'मदद',
 'hex': 'षोडश',
 'id': 'पहचान',
 'input': 'इनपुट',
 'int': 'अंक',
 'isinstance': 'उदाहरण_है',
 'issubclass': 'उपवर्ग_है',
 'iter': 'दोहरा',
 'len': 'लंबाई',
 'list': 'सूची',
 'locals': 'लोकल',
 'map': 'नक्शा',
 'max': 'अधिकतम',
 'memoryview': 'स्मृति_दर्शन',
 'min': 'न्यूनतम',
 'next': 'अगला',
 'object': 'चीज़',
 'oct': 'अष्ट',
 'open': 'खोल',
 'ord': 'क्रम',
 'pow': 'घात',
 'print': 'छापिये ',
 'property'

In [ ]:
class CodeDataset(Dataset):
    """Custom Dataset for code translation"""
    def __init__(self, codes):
        self.codes = codes
    
    def __len__(self):
        return len(self.codes)
    
    def __getitem__(self, idx):
        return self.codes[idx]

class GPUCodeTranslator:
    def __init__(self, keywords_dict, batch_size=32, device=None):
        self.translator = GoogleTranslator(source='en', target='hi')
        self.keywords = keywords_dict
        self.batch_size = batch_size
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Add common translations
        self.special_translations = {
            'i': 'ई',
            'j': 'जे',
            'k': 'के'
        }
        self.keywords.update(self.special_translations)
        print(f"Using device: {self.device}")
    
    def safe_translate(self, text, max_retries=3):
        """Translate text with retry mechanism"""
        if not text or not isinstance(text, str):
            return text
            
        for attempt in range(max_retries):
            try:
                time.sleep(0.2)  # Reduced delay for GPU processing
                return self.translator.translate(text)
            except Exception as e:
                if attempt == max_retries - 1:
                    return text
                time.sleep(0.5)
        return text
    
    def translate_line(self, line):
        """Translate a single line of code"""
        # print(f"\nTranslating line: {line}")  # Debug print
        indent = len(line) - len(line.lstrip())
        line = line.lstrip()
        
        if not line:
            return line
        
        try:
            # Handle comments
            if '#' in line:
                code_part, comment_part = line.split('#', 1)
                translated_comment = self.safe_translate(comment_part.strip())
                # print(f"Translated comment: {translated_comment}")  # Debug print
                
                if code_part:
                    tokens = re.findall(r'[a-zA-Z_]+|\d+|[^\w\s]|\s+', code_part)
                    translated_tokens = []
                    
                    for token in tokens:
                        if token.isspace():
                            translated_tokens.append(token)
                        elif token.isalpha():
                            translated = self.keywords.get(token, self.safe_translate(token))
                            # print(f"Token '{token}' translated to '{translated}'")  # Debug print
                            translated_tokens.append(translated)
                        else:
                            translated_tokens.append(token)
                    
                    translated_code = ''.join(translated_tokens)
                    return ' ' * indent + translated_code.rstrip() + ' #' + translated_comment
                else:
                    return ' ' * indent + '#' + translated_comment
            
            # Handle code-only lines
            tokens = re.findall(r'[a-zA-Z_]+|\d+|[^\w\s]|\s+', line)
            translated_tokens = []
            
            for token in tokens:
                if token.isspace():
                    translated_tokens.append(token)
                elif token.isalpha():
                    translated = self.keywords.get(token, self.safe_translate(token))
                    # print(f"Token '{token}' translated to '{translated}'")  # Debug print
                    translated_tokens.append(translated)
                else:
                    translated_tokens.append(token)
            
            return ' ' * indent + ''.join(translated_tokens)
            
        except Exception as e:
            print(f"Line translation error: {str(e)}")
            return line
    
    def translate_code(self, code):
        """Translate complete code block"""
        if not isinstance(code, str):
            return ""
        
        # Handle literal \n in the input
        if '\\n' in code:
            lines = code.strip("'\"").split('\\n')
            translated_lines = []
            
            for line in lines:
                translated_line = self.translate_line(line.strip())
                translated_lines.append(translated_line)
            
            return '\\n '.join(translated_lines)
        
        # Handle regular newlines
        lines = code.split('\n')
        translated_lines = []
        
        for line in lines:
            translated_line = self.translate_line(line)
            translated_lines.append(translated_line)
        
        return '\n'.join(translated_lines)
    
    def process_batch(self, batch):
        """Process a batch of code samples in parallel"""
        translated_batch = []
        
        # Move batch to GPU if available
        if isinstance(batch, torch.Tensor):
            batch = batch.to(self.device)
        
        for code in batch:
            if isinstance(code, torch.Tensor):
                code = code.cpu().numpy().item()
            translated_code = self.translate_code(code)
            translated_batch.append(translated_code)
        
        return translated_batch

def process_test_samples(input_path, output_path, keywords_dict, num_samples=10):
    """Process just a few samples as a test"""
    # Initialize translator
    translator = GPUCodeTranslator(keywords_dict, batch_size=5)  # Smaller batch size for testing
    
    try:
        # Load dataset and take first few samples
        df = pd.read_csv(input_path)
        test_df = df.head(num_samples)
        # print(f"Processing {num_samples} test samples")
        
        # Create dataset and dataloader
        dataset = CodeDataset(test_df['English_code'].tolist())
        dataloader = DataLoader(dataset, batch_size=5, shuffle=False)
        
        # Process batches
        translated_codes = []
        for batch in tqdm(dataloader, desc="Translating test samples"):
            translated_batch = translator.process_batch(batch)
            translated_codes.extend(translated_batch)
        
        # Create results DataFrame
        results_df = pd.DataFrame({
            'English_code': test_df['English_code'],
            'Hindi_code': translated_codes
        })
        
        # Save results
        results_df.to_csv(output_path, index=False)
        print(f"\nTest results saved to: {output_path}")
        
        # Display all translations
        print("\nAll Translations:")
        print("-" * 80)
        for idx, row in results_df.iterrows():
            print(f"\nSample {idx + 1}:")
            print("Original:")
            print(row['English_code'])
            print("\nTranslated:")
            print(row['Hindi_code'])
            print("-" * 80)
        
        return results_df
        
    except Exception as e:
        print(f"Error processing test samples: {str(e)}")
        return None



In [ ]:
def main():
    # Set up GPU memory management
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    # Load keywords
    keywords = load_keywords()
    
    # Set paths
    input_path = 'data/extracted_code_blocks.csv'
    output_path = 'data/code_translations.csv'
    
    # Create output directory
    os.makedirs('data', exist_ok=True)
    
    # Process test samples
    processed_df = process_test_samples(input_path, output_path, keywords, num_samples=100)
    
    if processed_df is not None:
        print("\nTest run completed successfully!")

if __name__ == "__main__":
    main()

Using device: cuda


Translating test samples:  30%|███       | 6/20 [05:57<14:08, 60.60s/it]